<a href="https://colab.research.google.com/github/homa-abedin/BigData/blob/main/spark_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyspark

In [ ]:
from pyspark import SparkContext


In [ ]:
from pyspark.sql.functions import *

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark =SparkSession.builder.master('local[*]').appName('Book RAting').getOrCreate()

In [ ]:
spark

In [ ]:
Books=spark.read.options(sep=';', header=True,inferSchema=True).csv('/content/Books.csv')

In [ ]:
Book_Ratings=spark.read.options(sep=';', header=True,inferSchema=True).csv('/content/Book-Ratings.csv')

In [ ]:
Users=spark.read.options(sep=';', header=True,inferSchema=True).csv('/content/Users.csv')

In [ ]:
Books.show(10)

+----------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|           BookTitle|          BookAuthor|YearOfPublication|           Publisher|           ImageURLS|           ImageURLM|           ImageURLL|
+----------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|             2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|             2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|             1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|    

In [ ]:
BookJoinRate= Books.join(Book_Ratings, Books.ISBN==Book_Ratings.isbn,  'inner').drop(Book_Ratings.isbn)

In [ ]:
BookJoinRate.show(10)

+----------+--------------------+----------------+-----------------+--------------------+--------------------+--------------------+--------------------+------+----+
|      ISBN|           BookTitle|      BookAuthor|YearOfPublication|           Publisher|           ImageURLS|           ImageURLM|           ImageURLL|userid|rate|
+----------+--------------------+----------------+-----------------+--------------------+--------------------+--------------------+--------------------+------+----+
|0001046438|                Liar|     Stephen Fry|                0|    Harpercollins Uk|http://images.ama...|http://images.ama...|http://images.ama...| 23902|   9|
|0001046934|The Prime of Miss...|    Muriel Spark|             1999|Trafalgar Square ...|http://images.ama...|http://images.ama...|http://images.ama...|206300|   0|
|0001047213|    The Fighting Man|  Gerald Seymour|             1993|HarperCollins Pub...|http://images.ama...|http://images.ama...|http://images.ama...| 23902|   9|
|000104797

In [ ]:
BookJoinRate.createOrReplaceTempView('rating')

# question 1

In [ ]:
avgrate= spark.sql('select isbn,BookTitle ,avg(rate)as avg from rating group by isbn, BookTitle ')

In [ ]:
avgrate.count()

154301

In [ ]:
Books.count()

154400

In [ ]:
avgrate.show(10)

+----------+--------------------+----+
|      isbn|           BookTitle| avg|
+----------+--------------------+----+
|0001046438|                Liar| 9.0|
|0001046934|The Prime of Miss...| 0.0|
|0001047213|    The Fighting Man| 9.0|
|0001047973|     Brave New World| 9.0|
|0001048473|Nothing Can Be Be...| 0.0|
|0001053744| Pearl and Sir Orfeo| 5.0|
|000123207X|Paddington's Birt...| 0.0|
|0001232088|Paddington in the...| 0.0|
|0001360469|       Babe Dressing|10.0|
|0001372564|       Which Colour?| 0.0|
+----------+--------------------+----+
only showing top 10 rows



#Question 2

In [ ]:
AllData =BookJoinRate . join (Users,BookJoinRate.userid==Users.UserID ,'inner'). drop (BookJoinRate.userid)

In [ ]:
AllData.count()

874801

In [ ]:
alldataavg=AllData.join (avgrate, avgrate.isbn== AllData.ISBN , 'inner').drop(avgrate.isbn , avgrate.BookTitle)

In [ ]:
alldataavg.orderBy('isbn').show(10)

+---------------+-----------------+--------------------+--------------------+--------------------+--------------------+----+------+-----------+--------------------+----+----------+--------------------+---+
|     BookAuthor|YearOfPublication|           Publisher|           ImageURLS|           ImageURLM|           ImageURLL|rate|UserID|   USERNAME|            Location| Age|      isbn|           BookTitle|avg|
+---------------+-----------------+--------------------+--------------------+--------------------+--------------------+----+------+-----------+--------------------+----+----------+--------------------+---+
|    Stephen Fry|                0|    Harpercollins Uk|http://images.ama...|http://images.ama...|http://images.ama...|   9| 23902|cHwJip4Kj4k|london, england, ...|NULL|0001046438|                Liar|9.0|
|Helen Forrester|             1992|HarperCollins Pub...|http://images.ama...|http://images.ama...|http://images.ama...|   0|196149|ABBmQTNgpez|lisburn, northern...|  29|0001046

In [ ]:
alldataavg.createOrReplaceTempView('alldataavg')

In [ ]:
final =spark.sql('select USERNAME ,BookTitle ,rate, avg , case when rate>avg then '"'up'"' when rate=avg then '"'same'"' else '"'down'"' end as grade  from alldataavg ')

In [ ]:
final.show(10)

+-----------+--------------------+----+----+-----+
|   USERNAME|           BookTitle|rate| avg|grade|
+-----------+--------------------+----+----+-----+
|cHwJip4Kj4k|                Liar|   9| 9.0| same|
|6VUiynjA3tV|The Prime of Miss...|   0| 0.0| same|
|cHwJip4Kj4k|    The Fighting Man|   9| 9.0| same|
|EM5BvtuvZ91|     Brave New World|   9| 9.0| same|
|cHwJip4Kj4k|     Brave New World|   9| 9.0| same|
|cHwJip4Kj4k|Nothing Can Be Be...|   0| 0.0| same|
|e8Hb8GyY8yN| Pearl and Sir Orfeo|   5| 5.0| same|
|3X2htHp8Jgk|Paddington's Birt...|   0| 0.0| same|
|3X2htHp8Jgk|Paddington in the...|   0| 0.0| same|
|gjo86LabiFL|       Babe Dressing|  10|10.0| same|
+-----------+--------------------+----+----+-----+
only showing top 10 rows



In [ ]:
final.groupBy('grade').count().show(10)

+-----+------+
|grade| count|
+-----+------+
| down|459504|
| same|127789|
|   up|287508|
+-----+------+



# ****So 459504 of rate is below mean , 127789 of rate is equal to the mean and 287508 of rate is above mean ****

In [ ]:
final.write.csv('/content/final.csv')